In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [5]:
import sklearn

In [6]:
sklearn.__version__

'1.0.2'

In [7]:
#Q1 Jan columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

Answer: Q1 is 18

In [8]:
df['tpep_pickup_datetime']

0         2024-01-01 00:57:55
1         2024-01-01 00:03:00
2         2024-01-01 00:17:06
3         2024-01-01 00:36:38
4         2024-01-01 00:46:51
                  ...        
2964619   2024-01-31 23:45:59
2964620   2024-01-31 23:13:07
2964621   2024-01-31 23:19:00
2964622   2024-01-31 23:07:23
2964623   2024-01-31 23:58:25
Name: tpep_pickup_datetime, Length: 2964624, dtype: datetime64[ns]

Q2

In [9]:
# Calculate the duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter data for January
january_trips = df[df['tpep_pickup_datetime'].dt.month == 1]

# Calculate the standard deviation of the duration
std_duration_january = january_trips['duration'].std()

std_duration_january


34.85113736077084

Q2 Asnwer is 32.59

Q3 

In [10]:
# Filter the duration to keep records between 1 and 60 minutes
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records left
fraction_left = len(filtered_df) / len(df)

fraction_left


0.9778326020432945

In [11]:
# Check the fraction
fraction_left * 100

97.78326020432945

Q3 Answer is 98%

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [13]:
from sklearn.feature_extraction import DictVectorizer

# Re-cast IDs to strings
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Create a list of dictionaries
data_dict = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
vec = DictVectorizer(sparse=True)
feature_matrix = vec.fit_transform(data_dict)

# Get the dimensionality of the feature matrix (number of columns)
num_columns = feature_matrix.shape[1]

num_columns


521

Q4 Asnwer is 515

Q5

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming 'duration' is the target variable
X = feature_matrix
y = df['duration']

# Train a plain linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict on the training data
y_pred = model.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

rmse

34.099551659298314

Q5 Asnwer is 16.64

Q6 

In [15]:
# Load your validation dataset
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')


In [16]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007526 entries, 0 to 3007525
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [17]:

# Convert start and end time to datetime
df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])

# Calculate the duration in minutes
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

# Re-cast IDs to strings
df_val['PULocationID'] = df_val['PULocationID'].astype(str)
df_val['DOLocationID'] = df_val['DOLocationID'].astype(str)

# Create a list of dictionaries for the validation data
data_dict_val = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform the validation data using the fitted vectorizer
X_val = vec.transform(data_dict_val)
y_val = df_val['duration']

# Predict on the validation data
y_val_pred = model.predict(X_val)

# Calculate RMSE for the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))

rmse_val


33.49222786499236

Q6 answer is 16.81